In [1]:
from DeepPurpose import utils, dataset
from DeepPurpose import DTI as models
import ast
import numpy as np
import json
import os
import requests
import warnings
warnings.filterwarnings("ignore")

In [2]:
kiba_data = dataset.load_process_KIBA()

Beginning Processing...
100% [............................................................................] 338300 / 338300Beginning to extract zip file...
Done!


In [3]:
file_obj = open("proteins.txt", 'r')
file = ast.literal_eval(file_obj.read())

In [4]:
print(kiba_data[0].__len__(), kiba_data[1].__len__(), kiba_data[2].__len__())

118254 118254 118254


In [5]:
kiba_data = np.stack((kiba_data[0], kiba_data[1], kiba_data[2]), axis=-1)

In [6]:
kiba_data.shape

(118254, 3)

In [17]:
result = []
for row in kiba_data:
    value = row[1]
    key = list(filter(lambda x: file[x] == value, file))[0] 
    new_row = np.append(row, key)
    result.append(new_row)
result = np.array(result)

In [20]:
import pandas as pd
kiba_data_df = pd.DataFrame(result, columns=['Smiles', 'molecules', 'target_affinity', 'uniprot_id'])

In [68]:
 kiba_data_df.to_csv('kiba_data_df_with_index_col.csv')

### Uniprot id to PDB id mapping

In [21]:
# get uniprot ids
keys = list(file.keys())

In [22]:
# load pdb id mappings
with open('idmapping_2023_11_12.json', 'r') as file:
    mappings = json.load(file)

In [58]:
pdb_ids = {}
c = 0
for i in range(len(mappings['results'])):
    if mappings['results'][i]['from'] in keys and mappings['results'][i]['from'] not in pdb_ids.keys(): # only take the first mapping
        pdb_ids[mappings['results'][i]['from']] = mappings['results'][i]['to']

In [61]:
mappings_keys = []
for i in range(len(mappings['results'])):
    mappings_keys.append(mappings['results'][i]['from'])

In [63]:
uniprot_ids = []
for k in keys:
    if k in mappings_keys:
        continue
    else:
        uniprot_ids.append(k)

In [65]:
print(pdb_ids.__len__(), uniprot_ids.__len__())

195 34


In [36]:
#EXECUTED - all succesfull
download_dir = 'PDB_files'
if not os.path.exists(download_dir):
    os.makedirs(download_dir)

# Base URL
base_url = 'https://files.rcsb.org/download/'

for pdb_id in pdb_ids.values():
    # Download URL with PDB ID
    download_url = f'{base_url}{pdb_id}.pdb'

    save_path = os.path.join(download_dir, f'{pdb_id}.pdb')

    # Make the request to download the file
    response = requests.get(download_url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        with open(save_path, 'wb') as file:
            file.write(response.content)
        #print(f'Successfully downloaded {pdb_id}.pdb')
    else:
        print(f'Failed to download {pdb_id}.pdb. Status code: {response.status_code}')


In [64]:
# DONE - DON'T EXECUTE AGAIN
# get pdb-files for proteins wihtout pdb id (Alphafold)

download_dir = 'PDB_files'

for uniprot_id in uniprot_ids:
    # Download URL with UNIPROT ID
    download_url = f'https://alphafold.ebi.ac.uk/files/AF-{uniprot_id}-F1-model_v2.pdb'

    save_path = os.path.join(download_dir, f'{uniprot_id}.pdb')

    # Make the request to download the file
    response = requests.get(download_url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        with open(save_path, 'wb') as file:
            file.write(response.content)
        #print(f'Successfully downloaded {uniprot_id}.pdb')
    else:
        print(f'Failed to download {uniprot_id}.pdb. Status code: {response.status_code}')